In [1]:
%pylab inline
import time 
import random
import pickle
import os
import sys
from functools import reduce
import numba

from pathlib import Path
sys.path.append("../src")

from IPython.display import Audio
from tqdm import tqdm, trange
import pandas as pd
import torch 
import miniaudio

%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%aimport -auto

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [107]:
import os
import eyed3
import pandas as pd

DATA_DIR = "/n4Ta/sc_webapp_data/"

In [113]:
fname = "mp3_r32_info_dct.pcl"
path = DATA_DIR + fname
if os.path.exists(path):
    mp3_r32_info_dct = pickle.load(open(path, "rb"))
    mp3_files = mp3_r32_info_dct["filepaths"]
else:
    user_path = "/r32Ta/soundcloud_music/files/"
    mp3_files = []
    for root, dirs, files in os.walk(user_path):
        for file in files:
            if file.endswith(".mp3"):
                mp3_files.append(os.path.join(root, file))
    mp3_r32_info_dct = {"filepaths": mp3_files}
    pickle.dump(mp3_r32_info_dct, open(path, "wb"))

In [77]:
eyed3.log.setLevel("ERROR")
def extract_mp3_info(path):
    try:
        audiofile = eyed3.load(path)
        
        if audiofile is not None and audiofile.tag is not None:
            genre_dct = {}
            if audiofile.tag.genre is None:
                genre_dct = {
                    'genre': None,
                    'genre_id': None,                
                }
                
            else:
                genre_dct = {
                    'genre': audiofile.tag.genre._name,
                    'genre_id': audiofile.tag.genre._id,                
                }
                
            
            return pd.Series(
                {
                    **{
                'title': audiofile.tag.title,
                'artist': audiofile.tag.artist,
                'album': audiofile.tag.album,
                'album artist': audiofile.tag.album_artist,
                'track number': audiofile.tag.track_num[0] if audiofile.tag.track_num else None,
                'release year': str(audiofile.tag.getBestDate())[:4] if audiofile.tag.getBestDate() else None,
                'duration (seconds)': audiofile.info.time_secs if audiofile.info else None,
                'bitrate (kbps)': audiofile.info.bit_rate[1] if audiofile.info and audiofile.info.bit_rate else None
                },
                    **genre_dct
                
                })
    except Exception as e:
        print(f"Error processing {path}: {e}")
    return pd.Series({
        'title': None, 'artist': None, 'album': None, 'album artist': None,
        'track number': None, 'genre': None, 'genre_id': None, 'release year': None, 'duration (seconds)': None,
        'bitrate (kbps)': None
    })


In [101]:
info_path = DATA_DIR+"df_with_mp3_info.tsv.gz"
if os.path.exists(info_path):
    w_info_df = pd.read_csv(path, sep="\t", compression="gzip")
else
    df = pd.DataFrame.from_dict({'paths': mp3_files}).astype({"paths": "string[pyarrow]"})
    df["filename"]=df["paths"].apply(lambda x: x.split("/")[-1].split(".mp3")[0])

    proc_df = df.head(1000)
    out = []
    for i, path in tqdm(enumerate(proc_df["paths"].values), total=len(proc_df)):
        out.append(extract_mp3_info(path))
    mp3_info_df =  pd.DataFrame(out)    
    w_info_df = pd.concat([proc_df, mp3_info_df], axis=1)
    w_info_df.to_csv(info_path , sep="\t", compression="gzip", index=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1041.27it/s]


In [127]:
w_info_df.columns

Index(['paths', 'filename', 'title', 'artist', 'album', 'album artist',
       'track number', 'release year', 'duration (seconds)', 'bitrate (kbps)',
       'genre', 'genre_id'],
      dtype='object')

In [128]:
w_info_df.head()

,paths,filename,title,artist,album,album artist,track number,release year,duration (seconds),bitrate (kbps),genre,genre_id
0,/r32Ta/soundcloud_music/files/user-337026238/C...,"Crown Imperial, by William Walton","Crown Imperial, by William Walton",Harrogate Symphony Orchestra,NaN,NaN,NaN,2019.0,385.87,128.0,Classical,32.0
1,/r32Ta/soundcloud_music/files/user-337026238/S...,Symphony No.7 in A - Beethoven,Symphony No.7 in A - Beethoven,Harrogate Symphony Orchestra,NaN,NaN,NaN,2022.0,2327.26,128.0,Classical,32.0
2,/r32Ta/soundcloud_music/files/user-337026238/H...,HSO 2019 Ripon Cathedral Concert(2nd Half),HSO 2019 Ripon Cathedral Concert(2nd Half),Harrogate Symphony Orchestra,NaN,NaN,NaN,2019.0,2581.49,128.0,Classical,32.0
3,/r32Ta/soundcloud_music/files/user-337026238/F...,"Fantasia On British Sea Songs, by Henry Wood ...","Fantasia On British Sea Songs, by Henry Wood /...",Harrogate Symphony Orchestra,NaN,NaN,NaN,2019.0,404.12,128.0,Classical,32.0
4,/r32Ta/soundcloud_music/files/user-337026238/P...,"Pomp & Circumstance March No. 1, Edward Elgar","Pomp & Circumstance March No. 1, Edward Elgar",Harrogate Symphony Orchestra,NaN,NaN,NaN,2019.0,388.62,128.0,Classical,32.0


In [140]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_path = DATA_DIR + "tfidf_feats.pckl"
if os.path.exists(tfidf_path):
    tfidf_features = pickle.load(open(tfidf_path, 'rb'))
else:
    tfidf_features = dict()
    for k in ["filename", "title", "artist"]:
        corpus = w_info_df[k].fillna("").values
        vectorizer = TfidfVectorizer()
        tfidf_features["X_"+k] = vectorizer.fit_transform(corpus)
        tfidf_features["vectorizer_"+k] = vectorizer
    pickle.dump(tfidf_features, open(tfidf_path, 'wb'))    

'vienna teng'

In [149]:
c = "artist"
k = 20
x_search = tfidf_features["vectorizer_" + c].transform(["vienna teng"])
out = np.array((tfidf_features["X_" + c].dot(x_search.T).T.todense()))[0]
idx_top_k = np.argsort(out)[::-1][:k]

Try to use LLM to sort in the best way here. Call Mistral/Mixtral --> Make prompt to create query with info. --> ask to sort in a way that makes the most sense.

In [151]:
w_info_df.loc[idx_top_k].sort_values(by=["artist", "release year", "track number"])

,paths,filename,title,artist,album,album artist,track number,release year,duration (seconds),bitrate (kbps),genre,genre_id
2112244,/r32Ta/soundcloud_music/files/nightowlcollecti...,workstudychill - Vol. 2_265_4540,4540,Vienna,work/study/chill - Vol. 2,NaN,265.0,2019.0,124.18,128.0,beat music,NaN
2112639,/r32Ta/soundcloud_music/files/nightowlcollecti...,workstudychill - Vol. 2_266_cortez,cortez,Vienna,work/study/chill - Vol. 2,NaN,266.0,2019.0,85.48,128.0,NaN,NaN
2113004,/r32Ta/soundcloud_music/files/nightowlcollecti...,workstudychill - Vol. 2_267_Demolition,Demolition,Vienna,work/study/chill - Vol. 2,NaN,267.0,2019.0,130.91,128.0,beat music,NaN
2112959,/r32Ta/soundcloud_music/files/nightowlcollecti...,workstudychill - Vol. 2_268_Tea (visuals in de...,Tea (visuals in description),Vienna,work/study/chill - Vol. 2,NaN,268.0,2019.0,109.41,128.0,beat music,NaN
2112795,/r32Ta/soundcloud_music/files/nightowlcollecti...,workstudychill - Vol. 2_270_printer,printer,Vienna,work/study/chill - Vol. 2,NaN,270.0,2019.0,85.43,128.0,beat music,NaN
1570984,/r32Ta/soundcloud_music/files/vienna-teng/song...,songwriting diaries_1_songwriting diary no. 1,songwriting diary no. 1,Vienna Teng,songwriting diaries,NaN,1.0,2012.0,414.44,128.0,NaN,NaN
1570985,/r32Ta/soundcloud_music/files/vienna-teng/song...,songwriting diaries_2_songwriting diary no. 2,songwriting diary no. 2,Vienna Teng,songwriting diaries,NaN,2.0,2012.0,757.30,128.0,NaN,NaN
1570978,/r32Ta/soundcloud_music/files/vienna-teng/song...,songwriting diaries_3_songwriting diary no. 3,songwriting diary no. 3,Vienna Teng,songwriting diaries,NaN,3.0,2012.0,365.92,128.0,NaN,NaN
1570972,/r32Ta/soundcloud_music/files/vienna-teng/the ...,the hymn of acxiom (1st draft demo),the hymn of acxiom (1st draft demo),Vienna Teng,NaN,NaN,NaN,2012.0,239.46,128.0,vocoder choir,NaN
1570977,/r32Ta/soundcloud_music/files/vienna-teng/Aims...,Aims_01_01 Level Up,01 Level Up,Vienna Teng,Aims,NaN,1.0,2013.0,234.43,128.0,Alternative,20.0


In [142]:
col="artist"

In [137]:
tfidf_features.keys()

dict_keys(['X_filename', 'vectorizer_filename', 'X_title', 'vectorizer_title', 'X_artist', 'vectorizer_artist'])

In [12]:
x_search = vectorizer.transform(["vienna teng"])

In [32]:
out = np.array((X.dot(x_search.T).T.todense()))[0]

In [38]:
top_10 = np.argsort(out)[::-1][:10]

In [39]:
out[top_10[0]]

0.6362747676903667

In [40]:
df.loc[top_10]

,paths,song_name
1998545,/r32Ta/soundcloud_music/files/the_fray/Vienna.mp3,Vienna
1334578,/r32Ta/soundcloud_music/files/larryfisherman/V...,Vienna
970729,/r32Ta/soundcloud_music/files/r0o0b/Vienna.mp3,Vienna
2057862,/r32Ta/soundcloud_music/files/hans-zimmer-offi...,Vienna
2145188,/r32Ta/soundcloud_music/files/vitamin-string-q...,Vienna
925837,/r32Ta/soundcloud_music/files/howard-shore-off...,Vienna
1640616,/r32Ta/soundcloud_music/files/interactivenoise...,Vienna
1788606,/r32Ta/soundcloud_music/files/dapayk/the vi-te...,the vi-teng clan
233643,/r32Ta/soundcloud_music/files/jazzinstrumental...,Vienna Jazz
544242,/r32Ta/soundcloud_music/files/hamburgradiosymp...,Vienna Blood


In [16]:
x_search.shape

(1, 1143919)

In [153]:
!ls

archive_tracker  copy_sc_data_v2.pckl	 music_info.tsv.gz
chromedriver	 music_info_tiny.tsv.gz  sc_data_v2.pckl


In [154]:
tiny_df = pd.read_csv("/home/keld/projects/music_gen/sc_scrape/data/music_info_tiny.tsv.gz", sep="\t", compression="gzip")

In [157]:
tiny_df.columns

Index(['album', 'albumartist', 'artist', 'audio_offset', 'bitrate', 'channels',
       'composer', 'disc', 'disc_total', 'duration', 'filesize', 'genre',
       'samplerate', 'title', 'track', 'track_total', 'year', 'local_path',
       'sc_user', 'sc_track_name', 'sc_link', 'tag', 'likes', 'reposts',
       'plays', 'comments', 'music_path', 'music_id'],
      dtype='object')

In [159]:
tiny_df.head()["music_path"]

0                        sony-classical/Benedictus.mp3
1    sony-classical/2CELLOS - Supermassive Black Ho...
2                   sony-classical/Highway To Hell.mp3
3         freemusicstreaming/Atlantis  Paul Landry.mp3
4        freemusicstreaming/Aphrodite  Paul Landry.mp3
Name: music_path, dtype: object

In [15]:
X.shape

(2198568, 1143919)

In [13]:
np.dot(X,x_search)

ValueError: dimension mismatch

In [ ]:
transform("vienna teng")

In [4]:
len(mp3_files)

2198568

In [14]:
#mp3_r32_info_dct

In [6]:
def score_terms(x, terms=None):
    if terms:
        terms = [term.lower() for term in terms]
        return sum([str(x).lower().find(term) != -1 for term in terms ])
    else: 
        return True

# scores = all_music_info_df["artist"].apply(lambda x: score_terms(x, ['Vienna', "Teng"]))

In [5]:

@numba.jit
def score_terms_numba(x, terms=None):
    if terms:
        terms = [term.lower() for term in terms]
        return sum([str(x).lower().find(term) != -1 for term in terms ])
    else: 
        return True

/tmp/ipykernel_330381/1792048388.py:4: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def score_terms_numba(x, terms=None):


In [9]:
%%timeit -n 1 -r 1
n = [fn + "_|_" if score_terms(fn, ["vienna", "teng"]) == 2 else '' for fn in mp3_files]
k = reduce(lambda x, y: x+y, n, '' )
k.split("_|_")[:-1]

13.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
%%timeit -n 1 -r 1
n = [fn + "_|_" if score_terms_numba(fn, ["vienna", "teng"]) == 2 else '' for fn in mp3_files]
k = reduce(lambda x, y: x+y, n, '' )
k.split("_|_")[:-1]


48.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
%%timeit -n 1 -r 1
search_df = df[df["paths"].apply(lambda x: score_terms(x, ["vienna", "teng"]) == 2)]

14.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
pd.__version__

'2.2.0'

In [ ]:
raw=True, engine_kwargs="parallel"

In [ ]:
roll.apply(f, engine='numba', raw=True)


In [15]:
f = lambda x: score_terms(x, ["vienna", "teng"]) == 2

ModuleNotFoundError: No module named 'sklearn'

In [25]:
df.head()

,paths,song_name
0,/r32Ta/soundcloud_music/files/user-337026238/C...,"Crown Imperial, by William Walton"
1,/r32Ta/soundcloud_music/files/user-337026238/S...,Symphony No.7 in A - Beethoven
2,/r32Ta/soundcloud_music/files/user-337026238/H...,HSO 2019 Ripon Cathedral Concert(2nd Half)
3,/r32Ta/soundcloud_music/files/user-337026238/F...,"Fantasia On British Sea Songs, by Henry Wood ..."
4,/r32Ta/soundcloud_music/files/user-337026238/P...,"Pomp & Circumstance March No. 1, Edward Elgar"


In [18]:
mp3_files[:10]

['/r32Ta/soundcloud_music/files/user-337026238/Crown Imperial, by William Walton.mp3',
 '/r32Ta/soundcloud_music/files/user-337026238/Symphony No.7 in A - Beethoven.mp3',
 '/r32Ta/soundcloud_music/files/user-337026238/HSO 2019 Ripon Cathedral Concert(2nd Half).mp3',
 '/r32Ta/soundcloud_music/files/user-337026238/Fantasia On British Sea Songs, by Henry Wood  Malcolm Sargent.mp3',
 '/r32Ta/soundcloud_music/files/user-337026238/Pomp & Circumstance March No. 1, Edward Elgar.mp3',
 '/r32Ta/soundcloud_music/files/user-337026238/Chanson de Matin, by Edward Elgar.mp3',
 '/r32Ta/soundcloud_music/files/user-337026238/Arutunian, Trumpet Concerto Extract.mp3',
 '/r32Ta/soundcloud_music/files/user-337026238/By The Sleepy Lagoon, by Eric Coates.mp3',
 '/r32Ta/soundcloud_music/files/user-337026238/Four Scottish Dances, by Malcolm Arnold.mp3',
 '/r32Ta/soundcloud_music/files/user-337026238/Elizabethan Serenade, by Ronald Binge.mp3']

In [17]:
%%timeit -n 1 -r 1
search_df = df[df["paths"].apply(f, engine='numba', raw=True)]

TypeError: <lambda>() got an unexpected keyword argument 'engine'

In [20]:
from functools import reduce
# k = sum(n)

In [35]:
search_df.dtypes

paths    object
dtype: object

In [36]:
pd.__version__

'2.1.0'

In [10]:
import pickle


In [8]:
def load_tags(filepath):
    audio_file = eyed3.load(filepath)
    return {'Filepath': filepath, 'Title': audio_file.tag.title, 'Artist': audio_file.tag.artist, 'Album': audio_file.tag.album}

data = {'Filepath': [], 'Title': [], 'Artist': [], 'Album': []}


In [9]:
load_tags()

{'Filepath': '/r32Ta/soundcloud_music/files/user-337026238/Crown Imperial, by William Walton.mp3',
 'Title': 'Crown Imperial, by William Walton',
 'Artist': 'Harrogate Symphony Orchestra',
 'Album': None}

In [10]:
fp = mp3_files[0]
audio_file = eyed3.load(fp)

In [20]:
tag_dict = {key: value for key, value in vars(audio_file.tag).items() if value is not None}


tag_dict

In [24]:
def frame_id_to_human_readable(frame_id):
    id_map = {
        'TIT2': 'Title',
        'TPE1': 'Artist',
        'TALB': 'Album',
        'TRCK': 'Track Number',
        'TLEN': 'Length',
        'TYER': 'Year',
        # Add more mappings based on ID3 standard
    }
    return id_map.get(frame_id, frame_id)

In [23]:
used_tags = []
for frame_id, frames in audio_file.tag.frame_set.items():
    used_tags.append(frame_id)

print(used_tags)

[b'TIT2', b'TPE1', b'TDRC', b'TCON', b'WOAR', b'COMM', b'APIC']


In [18]:
audio_file.tag.__dict__

{'file_info': <eyed3.id3.tag.FileInfo at 0x7f55818f31c0>,
 'header': <eyed3.id3.headers.TagHeader at 0x7f5581b53610>,
 'extended_header': <eyed3.id3.headers.ExtendedTagHeader at 0x7f55818d9810>,
 'frame_set': {b'TIT2': [<eyed3.id3.frames.TextFrame at 0x7f5581904f40>],
  b'TPE1': [<eyed3.id3.frames.TextFrame at 0x7f5581b52fe0>],
  b'TDRC': [<eyed3.id3.frames.DateFrame at 0x7f5581b53700>],
  b'TCON': [<eyed3.id3.frames.TextFrame at 0x7f5581b532b0>],
  b'WOAR': [<eyed3.id3.frames.UrlFrame at 0x7f5581999960>],
  b'COMM': [<eyed3.id3.frames.CommentFrame at 0x7f5581999930>],
  b'APIC': [<eyed3.id3.frames.ImageFrame at 0x7f5581999330>]},
 '_comments': <eyed3.id3.tag.CommentsAccessor at 0x7f55818f0e50>,
 '_images': <eyed3.id3.tag.ImagesAccessor at 0x7f55818f0c70>,
 '_lyrics': <eyed3.id3.tag.LyricsAccessor at 0x7f55818f2b60>,
 '_objects': <eyed3.id3.tag.ObjectsAccessor at 0x7f55818f1180>,
 '_privates': <eyed3.id3.tag.PrivatesAccessor at 0x7f5581b52ec0>,
 '_user_texts': <eyed3.id3.tag.UserTextsA

In [19]:
audio_file.tag.title

'Crown Imperial, by William Walton'

In [16]:
audio_file._tag.__dict__

{'file_info': <eyed3.id3.tag.FileInfo at 0x7f55818f31c0>,
 'header': <eyed3.id3.headers.TagHeader at 0x7f5581b53610>,
 'extended_header': <eyed3.id3.headers.ExtendedTagHeader at 0x7f55818d9810>,
 'frame_set': {b'TIT2': [<eyed3.id3.frames.TextFrame at 0x7f5581904f40>],
  b'TPE1': [<eyed3.id3.frames.TextFrame at 0x7f5581b52fe0>],
  b'TDRC': [<eyed3.id3.frames.DateFrame at 0x7f5581b53700>],
  b'TCON': [<eyed3.id3.frames.TextFrame at 0x7f5581b532b0>],
  b'WOAR': [<eyed3.id3.frames.UrlFrame at 0x7f5581999960>],
  b'COMM': [<eyed3.id3.frames.CommentFrame at 0x7f5581999930>],
  b'APIC': [<eyed3.id3.frames.ImageFrame at 0x7f5581999330>]},
 '_comments': <eyed3.id3.tag.CommentsAccessor at 0x7f55818f0e50>,
 '_images': <eyed3.id3.tag.ImagesAccessor at 0x7f55818f0c70>,
 '_lyrics': <eyed3.id3.tag.LyricsAccessor at 0x7f55818f2b60>,
 '_objects': <eyed3.id3.tag.ObjectsAccessor at 0x7f55818f1180>,
 '_privates': <eyed3.id3.tag.PrivatesAccessor at 0x7f5581b52ec0>,
 '_user_texts': <eyed3.id3.tag.UserTextsA